In [1]:
import base64
import hashlib
from Crypto.Cipher import AES
from Crypto import Random
import os

In [2]:
def pad(s):
    block_size = 16
    remainder = len(s) % block_size
    padding_needed = block_size - remainder
    return s + padding_needed * ' '

# remove the extra spaces at the end
def unpad(s): 
    return s.rstrip()
 
def encrypt(plain_text, password):
    # generate a random salt
    salt = os.urandom(AES.block_size)

    # generate a random iv
    iv = Random.new().read(AES.block_size)

    # use the Scrypt KDF to get a private key from the password
    private_key = hashlib.scrypt(password.encode(), salt=salt, n=2**14, r=8, p=1, dklen=32)

    # pad text with spaces to be valid for AES CBC mode
    padded_text = pad(plain_text)
    
    # create cipher config
    cipher_config = AES.new(private_key, AES.MODE_CBC, iv)

    # return a dictionary with the encrypted text
    return {
        'cipher_text': base64.b64encode(cipher_config.encrypt(padded_text)),
        'salt': base64.b64encode(salt),
        'iv': base64.b64encode(iv)
    }

In [5]:
print(encrypt('pass','123'))

{'cipher_text': b'Mrc5rHwzvHQPaiBJymqFEQ==', 'salt': b'wKb/XCRzyrMs+ldvSD2SCA==', 'iv': b'6PSvdxAxy7bMSjiHYYqnpA=='}


In [10]:
def decrypt(enc_dict, password):
    # decode the dictionary entries from base64
    salt = base64.b64decode(enc_dict['salt'])
    enc = base64.b64decode(enc_dict['cipher_text'])
    iv = base64.b64decode(enc_dict['iv'])

    # generate the private key from the password and salt
    private_key = hashlib.scrypt(password.encode(), salt=salt, n=2**14, r=8, p=1, dklen=32)

    # create the cipher config
    cipher = AES.new(private_key, AES.MODE_CBC, iv)

    # decrypt the cipher text
    decrypted = cipher.decrypt(enc)

    # unpad the text to remove the added spaces
    original = unpad(decrypted)

    return original

def main():
    password = input("Password: ")
    
    # First let us encrypt secret message
    encrypted = encrypt("hii", password)
    print(encrypted)
    
    # Let us decrypt using our original password
    decrypted = decrypt(encrypted, password)
    print(bytes.decode(decrypted))

In [11]:
main()

Password:  ll


{'cipher_text': b'EVBEOoFT5EKhIkxvB4d/ag==', 'salt': b'SW62YFm07e0ynTZ5QoD59w==', 'iv': b'XJnTNTASw9DhVa+ekS8Eig=='}
hii


In [12]:
s = b'bP5cm8YMOeGio4gT1fEeIQ=='

In [13]:
type(s)

bytes

In [14]:
print(s)

b'bP5cm8YMOeGio4gT1fEeIQ=='


In [16]:
len(s.decode('utf-8'))

24

In [4]:
import os

In [12]:
res = os.popen('echo "hi" | openssl enc -e -aes-256-cbc -a -salt -pbkdf2 -pass pass:\'123\'')

In [13]:
type(res)

os._wrap_close

In [14]:
res

In [15]:
res.read().strip()

'U2FsdGVkX18RkNMZwzFeYOTk9ND+qRkz1lTPYahaPIs='

In [5]:
def encryptThis(text, password):
    res = os.popen('echo "{0}" | openssl enc -e -aes-256-cbc -a -salt -pbkdf2 -pass pass:{1}'.format(text,password))
    return res

In [6]:
encryptThis('hi', '123').read()

'U2FsdGVkX1/ica2eVb3IwSqiPsFH2zxJNWO43INCVZ0=\n'

In [7]:
def decryptThis(text,password):
    res = os.popen('echo "{0}" | openssl enc -e -aes-256-cbc -a -d -salt -pbkdf2 -pass pass:{1}'.format(text,password))
    return res

In [8]:
decryptThis('U2FsdGVkX1/Wm+v7NTwxGSBRDdI9/VnFtAlQXKIiFtg=','123').read()

'hi\n'